In [7]:
import requests
from selenium import webdriver
import re
import os
import csv
import pandas as pd
import datetime
import json
driver = webdriver.Firefox()

In [8]:
mp_list, day_list, time_list, month_list, year_list = [], [], [], [], []

def startdate(startmonth, startyear, endmonth, endyear):
    end = endmonth + ' ' + endyear
    start = driver.get("https://www.moongiant.com/moonphases/" + startmonth + '/' + startyear)
    def collectdata():
        phases = driver.find_elements_by_id('tablePhases')
        for i in phases:
            split_list = i.text.split()
                
        def chunker(seq, size):
            return (seq[pos:pos + size] for pos in range(0, len(seq), size))       
        for element in chunker(split_list, 6):
            if re.search(r'(\D*)(\d+)', element[2]): 
                mo = list(filter(None, re.split('(\d+)',element[2])))
                element[2:3] = mo
            joined = ' '.join(element[0:2])
            mp_list.append(joined)
            day_list.append(element[3])
            time_list.append(element[4])
            month_list.append(element[2])

        monthyear = driver.find_elements_by_css_selector('#placePhase > div:nth-child(1) > h1:nth-child(3)')
        for month in monthyear:
            year_pattern = re.compile(r'\d{4}')
            year_matches = year_pattern.findall(month.text)
            for match in year_matches:
                for element in chunker(split_list, 6):
                    year_list.append(match)
        return checkdate()
        
    def nextpage():
        driver.find_element_by_xpath('//*[@id="nextMonth"]').click()
        collectdata()
    def checkdate():
        monthyear = driver.find_elements_by_css_selector('#placePhase > div:nth-child(1) > h1:nth-child(3)')
        for month in monthyear:
            if end not in month.text:
                nextpage()
            else:
                break
                
    collectdata()
    checkdate()
    
startdate('January', '2016', 'December', '2017')

final_array = []

for mp, day, time, month, year in zip(mp_list, day_list, time_list, month_list, year_list):
    final_array.append({'Moon Phase':mp, 'Day':day, 'Time (UTC)':time, 'Month': month, 'Year':year})

df = pd.DataFrame(final_array)
df['Time (UTC)'] = pd.to_datetime(df['Time (UTC)'], format='%H:%M').dt.time
df['Time (UTC)'] = (df['Time (UTC)'].apply(lambda t: t.strftime('%H:%M')))
df.to_csv('Moon_Data.csv')
df
driver.quit()

In [ ]:
df